<a href="https://colab.research.google.com/github/edmgjr/IFSP-CMP-D2APR-2021.2/blob/main/projects/portfolio/project-01_regression_California-housing-prices/california-housing-sprint-04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas) <br/>
**Prof**: Samuel Martins (Samuka) <br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>. <br/><br/>

### Custom CSS style

In [2]:
%%html
<style>
.dashed-box {
    border: 1px dashed black !important;
}
.dashed-box tr {
  background-color: white !important;  
}
.alt-tab {
    background-color: black;
    color: #ffc351;
    padding: 4px;
    font-size: 1em;
    font-weight: bold;
    font-family: monospace;
}
// add your CSS styling here
</style>

<span style='font-size: 2.5em'><b>California Housing 🏡</b></span><br/>
<span style='font-size: 1.5em'>Predict the median housing price in California districts</span>

<span style="background-color: #ffc351; padding: 4px; font-size: 1em;"><b>Sprint 4</b></span>

<img src="https://github.com/edmgjr/IFSP-CMP-D2APR-2021.2/blob/main/projects/portfolio/project-01_regression_California-housing-prices/imgs/california-flag.png?raw=1" width=300/>

---



## Before starting this notebook
This jupyter notebook is designed for **experimental and teaching purposes**. <br/>
Although it is (relatively) well organized, it aims at solving the _target problem_ by evaluating (and documenting) _different solutions_ for somes steps of the **machine learning pipeline** — see the ***Machine Learning Project Checklist by xavecoding***. <br/>
We tried to make this notebook as literally a _notebook_. Thus, it contains notes, drafts, comments, etc.<br/>

For teaching purposes, some parts of the notebook may be _overcommented_. Moreover, to simulate a real development scenario, we will divide our solution and experiments into **"sprints"** in which each sprint has some goals (e.g., perform _feature selection_, train more ML models, ...). <br/>
The **sprint goal** will be stated at the beginning of the notebook.

A ***final notebook*** (or any other kind of presentation) that compiles and summarizes all sprints — the target problem, solutions, and findings — should be created later.

#### Conventions

<ul>
    <li>💡 indicates a tip. </li>
    <li> ⚠️ indicates a warning message. </li>
    <li><span class='alt-tab'>alt tab</span> indicates and an extra content (<i>e.g.</i>, slides) to explain a given concept.</li>
</ul>

---

## 🎯 Sprint Goals
- Refactor our codes by using the sklearn Pipelines
- Evaluate the models in the Test Set
- Compare the models with the baseline
---

### 0. Imports and default settings for plotting

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

## 🛠️ 5. Prepare the Data

#### **Preprocessing tasks**
- Fill in missing values (imputation)
- Add new features
- Feature Scaling
- One-Hot Encoding

<table align="left" class="dashed-box">
<tr>
    <td><span class='alt-tab'>alt tab</span></td>
    <td><b>Slides:</b> Scikit-Learn Design Principles - Hyperparameters vs Parameters<br/>
        <b>Slides:</b> Scikit-Learn Design Principles - Main APIs</td>
</tr>
</table><br/><br/>

### 5.1. Load the cleaned training set

Let's consider the training and testing sets already cleaned (sprint #2):
- Drop duplicated instances (not found)
- Drop instances with `housing_median_age` capped at 52
- Drop instances with `median_house_value` capped at 500001.0

In [4]:
raw_url = 'https://raw.githubusercontent.com/edmgjr/IFSP-CMP-D2APR-2021.2/main/projects/portfolio/project-01_regression_California-housing-prices/datasets/housing_train_sprint-2.csv'

In [5]:
# load the cleaned training set
housing_train = pd.read_csv(raw_url)

In [6]:
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062,340900.0,INLAND
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683,309200.0,<1H OCEAN
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657,322300.0,NEAR BAY
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740,394400.0,NEAR OCEAN
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765,339500.0,<1H OCEAN


In [7]:
housing_train.shape

(14857, 10)

### 5.2. Separate the _features_ and the _target outcome_

In [8]:
housing_train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [9]:
# store the target outcome into a numpy array
y_train = housing_train['median_house_value'].values

In [10]:
y_train

array([340900., 309200., 322300., ..., 112500.,  88100.,  89000.])

In [11]:
y_train.shape

(14857,)

In [12]:
# overwrite the dataframe with only the features  
housing_train = housing_train.drop(columns=['median_house_value'])

In [13]:
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062,INLAND
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683,<1H OCEAN
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657,NEAR BAY
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740,NEAR OCEAN
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765,<1H OCEAN


In [14]:
housing_train.shape

(14857, 9)

### 5.3. Separate the _numerical_ and _categorical_ features
Since we perform different preprocessing tasks (transformations) to _numerical_ features and _categorical_ ones, let's split them into two different dataframes.

In [15]:
housing_train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [16]:
# numerical atributes
num_attributes = housing_train.columns.drop('ocean_proximity')
num_attributes

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [17]:
# categorical attributes
cat_attributes = ['ocean_proximity']
cat_attributes

['ocean_proximity']

In [18]:
# separating the features
housing_train_num = housing_train[num_attributes]
housing_train_cat = housing_train[cat_attributes]

In [19]:
housing_train_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765


In [20]:
housing_train_cat.head()

,ocean_proximity
0,INLAND
1,<1H OCEAN
2,NEAR BAY
3,NEAR OCEAN
4,<1H OCEAN


### 5.4. Filling in missing values

`sklearn.impute.SimpleImputer` <br/>
https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [21]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
imputer.fit(housing_train_num)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [22]:
imputer.statistics_  # computed medians

array([-118.45  ,   34.24  ,   27.    , 2142.    ,  441.    , 1207.    ,
        416.    ,    3.4559])

In [23]:
housing_train_num.median()

longitude             -118.4500
latitude                34.2400
housing_median_age      27.0000
total_rooms           2142.0000
total_bedrooms         441.0000
population            1207.0000
households             416.0000
median_income            3.4559
dtype: float64

<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td>The <code>SimpleImputer</code> finds out the <i>statistic for imputation</i> <b>for ALL features</b>.</td>
</tr>
<tr>
    <td></td>
    <td>We can save this <i>transformer</i> on the disk for future transfomations.</td>
</tr>
</table><br/><br/>

In [24]:
# filling in the missing values FOR ALL attributes
# it generates a numpy array
housing_train_num_imputed = imputer.transform(housing_train_num)
housing_train_num_imputed

array([[-1.2137e+02,  3.7060e+01,  2.5000e+01, ...,  3.0000e+02,
         1.0400e+02,  3.8062e+00],
       [-1.1839e+02,  3.4140e+01,  1.9000e+01, ...,  2.0210e+03,
         9.6000e+02,  5.5683e+00],
       [-1.2207e+02,  3.7410e+01,  2.6000e+01, ...,  8.1500e+02,
         2.1800e+02,  5.7657e+00],
       ...,
       [-1.2186e+02,  3.7310e+01,  2.4000e+01, ...,  1.8080e+03,
         6.2500e+02,  2.2259e+00],
       [-1.2132e+02,  3.7960e+01,  4.6000e+01, ...,  9.7500e+02,
         3.7300e+02,  2.0398e+00],
       [-1.1730e+02,  3.4140e+01,  3.9000e+01, ...,  8.4100e+02,
         3.2000e+02,  1.9432e+00]])

### 5.5. Adding new features
To _automate data preprocessing_ via sklearn, we will need _to create_ our **own transformer** to add the new features considered.

In [25]:
# template to create an own estimation
from sklearn.base import BaseEstimator, TransformerMixin


class NameOfYourTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X):
        return None  # return the transformed data instead of None


Since our custom transformer can be executed before other transformation, we will consider that the input is a **numpy 2D array**, not a _dataframe_. <br/>

This transformer will create 3 new features, based on the current ones:
- `total_rooms`
- `total_bedrooms`
- `population`
- `households`


Thus, we need to find their column indices first because our input will be a **numpy 2D array**.

In [26]:
# get the integer index of each attribute/column:
for index, column_name in enumerate(housing_train_num.columns):
    print(f'{index} = {column_name}')

0 = longitude
1 = latitude
2 = housing_median_age
3 = total_rooms
4 = total_bedrooms
5 = population
6 = households
7 = median_income


In [27]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

total_rooms_col_idx = 3
total_bedrooms_col_idx = 4
population_col_idx = 5
households_col_idx = 6


class HousingFeatEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X):
        n_rows = X.shape[0]
        
        # creating the new features
        rooms_per_household = X[:, total_rooms_col_idx] / X[:, households_col_idx]
        bedrooms_per_room = X[:, total_bedrooms_col_idx] / X[:, total_rooms_col_idx]
        population_per_household = X[:, population_col_idx] / X[:, households_col_idx]
        
        # to concatenate the new arrays as columns in our feature matrix, we need to reshape first
        rooms_per_household = rooms_per_household.reshape((n_rows, 1))
        bedrooms_per_room = bedrooms_per_room.reshape((n_rows, 1))
        population_per_household = population_per_household.reshape((n_rows, 1))
        
        # concatenating the new features into the feature matrix X
        X_out = np.hstack((X, rooms_per_household, bedrooms_per_room, population_per_household))
        
        return X_out


In [28]:
feat_engineer = HousingFeatEngineering()

housing_train_num_new_feats = feat_engineer.transform(housing_train_num.values)  # we need to convert it to numpy first
housing_train_num_new_feats

array([[-121.37      ,   37.06      ,   25.        , ...,    4.55769231,
           0.19409283,    2.88461538],
       [-118.39      ,   34.14      ,   19.        , ...,    5.2875    ,
           0.2037037 ,    2.10520833],
       [-122.07      ,   37.41      ,   26.        , ...,    5.43119266,
           0.19003378,    3.73853211],
       ...,
       [-121.86      ,   37.31      ,   24.        , ...,    3.1024    ,
           0.3362558 ,    2.8928    ],
       [-121.32      ,   37.96      ,   46.        , ...,    4.91152815,
           0.19923581,    2.61394102],
       [-117.3       ,   34.14      ,   39.        , ...,    5.565625  ,
           0.18809657,    2.628125  ]])

In [29]:
housing_train_num_new_feats.shape

(14857, 11)

In [30]:
# show the new feats
housing_train_num_new_feats[:, -3:]

array([[4.55769231, 0.19409283, 2.88461538],
       [5.2875    , 0.2037037 , 2.10520833],
       [5.43119266, 0.19003378, 3.73853211],
       ...,
       [3.1024    , 0.3362558 , 2.8928    ],
       [4.91152815, 0.19923581, 2.61394102],
       [5.565625  , 0.18809657, 2.628125  ]])

### 5.6. Feature Scaling
Exactly as performed in the previous sprint: **RobustScaler**. <br/>
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html

In [31]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(housing_train_num)

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)

In [32]:
housing_train_num_scaled = scaler.transform(housing_train_num)
housing_train_num_scaled

array([[-0.81337047,  0.752     , -0.11111111, ..., -0.93989637,
        -0.94545455,  0.1689414 ],
       [ 0.01671309, -0.02666667, -0.44444444, ...,  0.84352332,
         1.64848485,  1.01876055],
       [-1.00835655,  0.84533333, -0.05555556, ..., -0.40621762,
        -0.6       ,  1.1139619 ],
       ...,
       [-0.94986072,  0.81866667, -0.16666667, ...,  0.62279793,
         0.63333333, -0.5931999 ],
       [-0.7994429 ,  0.992     ,  1.05555556, ..., -0.24041451,
        -0.13030303, -0.68295153],
       [ 0.32033426, -0.02666667,  0.66666667, ..., -0.37927461,
        -0.29090909, -0.72953943]])

### 5.7. Categorical Variable Encoding
Instead of using the method `.get_dummies()` from _pandas_, let's use a method from _sklearn_.

`sklearn.preprocessing.OneHotEncoder` <br/>
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [33]:
housing_train_cat

,ocean_proximity
0,INLAND
1,<1H OCEAN
2,NEAR BAY
3,NEAR OCEAN
4,<1H OCEAN
...,...
14852,NEAR OCEAN
14853,<1H OCEAN
14854,<1H OCEAN
14855,INLAND


In [34]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
housing_train_cat_1hot = encoder.fit_transform(housing_train_cat)

In [35]:
housing_train_cat

,ocean_proximity
0,INLAND
1,<1H OCEAN
2,NEAR BAY
3,NEAR OCEAN
4,<1H OCEAN
...,...
14852,NEAR OCEAN
14853,<1H OCEAN
14854,<1H OCEAN
14855,INLAND


In [36]:
housing_train_cat_1hot

<14857x5 sparse matrix of type '<class 'numpy.float64'>'
	with 14857 stored elements in Compressed Sparse Row format>

<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td>Notice that the output is a <i>SciPy sparse matrix</i>, instead of a <i>NumPy array</i>. This is very useful when you have categorical attributes with <b>thousands of categories</b>.</td>
</tr>
<tr>
    <td></td>
    <td>After one-hot encoding, we get a matrix with thousands of columns, and the matrix is <i>full of 0s</i> except for <i>a single <b>1</b> per row</i>.</td>
</tr>
<tr>
    <td></td>
    <td>Using up tons of memory mostly to store zeros would be very wasteful, so instead a sparse matrix only stores the location of the nonzero elements.</td>
</tr>
</table><br/><br/>


In [37]:
# converting to NumPy array
housing_train_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [38]:
# getting the list of categories
encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

### 5.8. Creating Preprocessing `Pipelines`
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

<table align="left" class="dashed-box">
<tr>
    <td><span class='alt-tab'>alt tab</span></td>
    <td><b>Slides:</b> Scikit-Learn Design Principles - Pipelines<br/></td>
</tr>
</table><br/><br/>

Let's create a **Preprocessing `Pipeline`**.

In [39]:
from sklearn.pipeline import Pipeline

#### Pipeline for numerical data

In [40]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('feat_engineering', HousingFeatEngineering()),
    ('robust_scaler', RobustScaler())
])

In [41]:
housing_train_num_preprocessed = num_pipeline.fit_transform(housing_train_num)

In [42]:
housing_train_num_preprocessed

array([[-0.81337047,  0.752     , -0.11111111, ..., -0.41720345,
        -0.15307229,  0.03593627],
       [ 0.01671309, -0.02666667, -0.44444444, ...,  0.05044793,
        -0.00235622, -0.88393775],
       [-1.00835655,  0.84533333, -0.05555556, ...,  0.14252434,
        -0.21672549,  1.04374832],
       ...,
       [-0.94986072,  0.81866667, -0.16666667, ..., -1.34973604,
         2.07630238,  0.04559594],
       [-0.7994429 ,  0.992     ,  1.05555556, ..., -0.19046999,
        -0.07242097, -0.2835198 ],
       [ 0.32033426, -0.02666667,  0.66666667, ...,  0.22866686,
        -0.24710446, -0.26677954]])

In [ ]:
housing_train_num_preprocessed.shape

(14857, 11)

#### Pipeline for categorical data

In [43]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('one-hot-encoding', OneHotEncoder(handle_unknown='ignore'))
])

In [44]:
housing_train_cat_preprocessed = cat_pipeline.fit_transform(housing_train_cat)

In [45]:
housing_train_cat_preprocessed.toarray()

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [47]:
housing_train_cat_1hot

<14857x5 sparse matrix of type '<class 'numpy.float64'>'
	with 14857 stored elements in Compressed Sparse Row format>

In [48]:
np.all(housing_train_cat_preprocessed.toarray() == housing_train_cat_1hot)

True

### 5.9. Putting it all by `ColumnTransformer`
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

Applies _transformers_ to **columns** of an array or pandas DataFrame. <br/>
This **estimator** allows _different columns_ or _column subsets_ of the input to be **transformed *separately*** and the _features generated_ by each transformer will be _concatenated_ to form a **single feature space**. <br/>

This is useful for _heterogeneous or columnar data_, to combine several feature extraction mechanisms or transformations into a single transformer.

In [49]:
housing_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062,INLAND
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683,<1H OCEAN
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657,NEAR BAY
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740,NEAR OCEAN
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
14852,-124.14,41.06,32.0,1020.0,215.0,421.0,198.0,3.0208,NEAR OCEAN
14853,-118.22,33.98,32.0,2643.0,737.0,2784.0,711.0,2.5352,<1H OCEAN
14854,-121.86,37.31,24.0,1939.0,652.0,1808.0,625.0,2.2259,<1H OCEAN
14855,-121.32,37.96,46.0,1832.0,365.0,975.0,373.0,2.0398,INLAND


In [50]:
num_attributes

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [51]:
cat_attributes

['ocean_proximity']

In [52]:
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062,INLAND
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683,<1H OCEAN
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657,NEAR BAY
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740,NEAR OCEAN
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765,<1H OCEAN


In [53]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('feat_engineering', HousingFeatEngineering()),
    ('robust_scaler', RobustScaler())
])

cat_pipeline = Pipeline([
    ('one-hot-encoding', OneHotEncoder(handle_unknown='ignore'))
])


# (name, transformer, columns)
preprocessed_pipeline = ColumnTransformer([
    ('numerical', num_pipeline, num_attributes),
    ('categorical', cat_pipeline, cat_attributes)
])

<table align="left" class="dashed-box">
<tr>
    <td>⚠️</td>
    <td><b>BE CAREFUL</b>.</td>
</tr>
<tr>
    <td></td>
    <td>When performing the pipeline <i>"numerical"</i>, <code>ColumnTransformer</code> first <i>selects/filters</i> the columns passed by the list <code>num_attributes</code>. We then have a <i>new dataframe</i> with <b>new indices</b> that will be processed.</td>
</tr>
<tr>
    <td></td>
    <td>When generating new features, our custom transformer <code>HousingFeatEngineering()</code> assumes a given values for the indices of <code>total_rooms</code>, <code>total_bedrooms</code>, etc.</td>
</tr>
<tr>
    <td></td>
    <td>These considered indices <b>MUST MATCH EXACTLY</b> with the <i>corresponding columns</i> of the numpy array or dataframe passed as input. For our case, this matching is true.</td>
</tr>
<tr>
    <td></td>
    <td>But, <b>BE CAREFUL!!!</b></td>
</tr>
</table><br/><br/>

In [54]:
housing_train_pre_npy = preprocessed_pipeline.fit_transform(housing_train)

In [55]:
housing_train_pre_npy

array([[-0.81337047,  0.752     , -0.11111111, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01671309, -0.02666667, -0.44444444, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00835655,  0.84533333, -0.05555556, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.94986072,  0.81866667, -0.16666667, ...,  0.        ,
         0.        ,  0.        ],
       [-0.7994429 ,  0.992     ,  1.05555556, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32033426, -0.02666667,  0.66666667, ...,  0.        ,
         0.        ,  0.        ]])

In [56]:
housing_train_pre_npy.shape

(14857, 16)

In [57]:
preprocessed_pipeline.named_transformers_

{'categorical': Pipeline(memory=None,
          steps=[('one-hot-encoding',
                  OneHotEncoder(categories='auto', drop=None,
                                dtype=<class 'numpy.float64'>,
                                handle_unknown='ignore', sparse=True))],
          verbose=False), 'numerical': Pipeline(memory=None,
          steps=[('imputer',
                  SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                missing_values=nan, strategy='median',
                                verbose=0)),
                 ('feat_engineering', HousingFeatEngineering()),
                 ('robust_scaler',
                  RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                               with_centering=True, with_scaling=True))],
          verbose=False)}

In [58]:
preprocessed_pipeline.transformers_

[('numerical', Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='median',
                                 verbose=0)),
                  ('feat_engineering', HousingFeatEngineering()),
                  ('robust_scaler',
                   RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                                with_centering=True, with_scaling=True))],
           verbose=False), Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
         'total_bedrooms', 'population', 'households', 'median_income'],
        dtype='object')),
 ('categorical', Pipeline(memory=None,
           steps=[('one-hot-encoding',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True))],
          

### 5.10. Saving the Preprocessed Pipeline

In [60]:
import joblib

!mkdir models
joblib.dump(preprocessed_pipeline, '/content/models/preprocessed_pipeline.pkl')

mkdir: cannot create directory ‘models’: File exists


['/content/models/preprocessed_pipeline.pkl']

In [62]:
# to load the pipeline
loaded_preprocessed_pipeline = joblib.load('/content/models/preprocessed_pipeline.pkl')

In [63]:
housing_train_pre_npy_2 = loaded_preprocessed_pipeline.transform(housing_train)
housing_train_pre_npy_2

array([[-0.81337047,  0.752     , -0.11111111, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01671309, -0.02666667, -0.44444444, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00835655,  0.84533333, -0.05555556, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.94986072,  0.81866667, -0.16666667, ...,  0.        ,
         0.        ,  0.        ],
       [-0.7994429 ,  0.992     ,  1.05555556, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32033426, -0.02666667,  0.66666667, ...,  0.        ,
         0.        ,  0.        ]])

In [64]:
np.all(housing_train_pre_npy == housing_train_pre_npy_2)

True

### 5.11. Saving the Preprocessed Training Set

In [65]:
!mkdir datasets
np.save('./datasets/housing_train_pre_numpy_sprint-4.npy', housing_train_pre_npy)

## 🏋️‍♀️ 6. Train ML Algorithms

### 6.1. Getting the independent (features) and dependent variables (outcome)

In [66]:
X_train = housing_train_pre_npy
# we already have y_train

In [67]:
X_train.shape

(14857, 16)

In [68]:
y_train.shape

(14857,)

### 6.2. Training the Models

<h3 style="color: #ff5757 !important"><b>Cross-validation</b></h3>

#### **→ Linear Regression**

In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()  # default parameters
lin_scores = cross_val_score(lin_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)

In [70]:
# printing function
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
display_scores(lin_rmse_scores)

Scores: [58883.49218279 55293.5735956  55181.52250607 57775.16025404
 60155.15922657 59588.9688012  57781.68607191 59995.21362185
 59923.51235138 59132.12479715]
Mean: 58371.04134085788
Standard deviation: 1757.914437100754


<br/>

We have exactly the results of Sprint #3.
- **Linear Regression:** \\$58,371.04 ± \$1,757.91

#### Training the final model
After cross-validation, we can train our models by using the **entire** _training set_.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [ ]:
### saving the model
import joblib
joblib.dump(lin_reg, './models/linear_regression_sprint-04.pkl')

['./models/linear_regression_sprint-04.pkl']

## 🔬🧪 7. Evaluation on the Test Set

<table align="left" class="dashed-box">
<tr>
    <td>⚠️</td>
    <td>We should to evaluate <b>many other</b> <i>quick-and-dirty models</i> before any evaluation on the test set.</td>
</tr>
<tr>
    <td></td>
    <td>The strategy is to select the <i>most promising models</i> and <i>fine-tune them</i> (e.g., perform grid-search to find the best hyperparameters and/or try ensemble methods. The selected models could then be evaluated in the test set.</td>
</tr>
<tr>
    <td></td>
    <td>We opted for evaluating our single linear regression model just to complete the end-to-end pipeline in these early sprints. We will perform the above strategy in the next sprints.</td>
</tr>
</table><br/><br/>

### 7.1. Prepare the Data

In [ ]:
### Load the testing set
housing_test = pd.read_csv('./datasets/housing_test_sprint-2.csv')
housing_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-118.25,34.00,41.0,1768.0,475.0,1721.0,474.0,1.3030,90400.0,<1H OCEAN
1,-118.20,33.99,35.0,1705.0,523.0,2252.0,508.0,2.3421,154200.0,<1H OCEAN
2,-121.39,39.61,22.0,2828.0,610.0,986.0,391.0,2.8871,94700.0,INLAND
3,-118.46,34.00,39.0,4098.0,1100.0,2054.0,1053.0,2.9180,345600.0,<1H OCEAN
4,-121.03,39.14,10.0,3138.0,524.0,1275.0,511.0,4.0775,164500.0,INLAND
...,...,...,...,...,...,...,...,...,...,...
3710,-122.05,37.36,34.0,2400.0,419.0,1017.0,384.0,4.1369,316900.0,<1H OCEAN
3711,-118.16,34.02,42.0,814.0,216.0,773.0,208.0,2.5313,156900.0,<1H OCEAN
3712,-120.65,35.27,15.0,2365.0,538.0,1446.0,490.0,2.5129,225900.0,NEAR OCEAN
3713,-118.32,33.94,37.0,2740.0,504.0,1468.0,479.0,4.5368,168800.0,<1H OCEAN


In [ ]:
### Separate the _features_ and the _target outcome_
y_test = housing_test['median_house_value'].values
y_test

array([ 90400., 154200.,  94700., ..., 225900., 168800., 220100.])

In [ ]:
housing_test = housing_test.drop(columns=['median_house_value'])
housing_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-118.25,34.00,41.0,1768.0,475.0,1721.0,474.0,1.3030,<1H OCEAN
1,-118.20,33.99,35.0,1705.0,523.0,2252.0,508.0,2.3421,<1H OCEAN
2,-121.39,39.61,22.0,2828.0,610.0,986.0,391.0,2.8871,INLAND
3,-118.46,34.00,39.0,4098.0,1100.0,2054.0,1053.0,2.9180,<1H OCEAN
4,-121.03,39.14,10.0,3138.0,524.0,1275.0,511.0,4.0775,INLAND
...,...,...,...,...,...,...,...,...,...
3710,-122.05,37.36,34.0,2400.0,419.0,1017.0,384.0,4.1369,<1H OCEAN
3711,-118.16,34.02,42.0,814.0,216.0,773.0,208.0,2.5313,<1H OCEAN
3712,-120.65,35.27,15.0,2365.0,538.0,1446.0,490.0,2.5129,NEAR OCEAN
3713,-118.32,33.94,37.0,2740.0,504.0,1468.0,479.0,4.5368,<1H OCEAN


In [ ]:
### Preprocess the Test Set
import joblib
preprocessed_pipeline = joblib.load('./models/preprocessed_pipeline.pkl')

# preprocess the test set
X_test = preprocessed_pipeline.transform(housing_test)

In [ ]:
X_test

array([[ 0.05571031, -0.064     ,  0.77777778, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.06963788, -0.06666667,  0.44444444, ...,  0.        ,
         0.        ,  0.        ],
       [-0.8189415 ,  1.432     , -0.27777778, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.61281337,  0.27466667, -0.66666667, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.0362117 , -0.08      ,  0.55555556, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02228412,  0.056     , -0.94444444, ...,  0.        ,
         0.        ,  0.        ]])

### 7.2. Prediction

In [ ]:
### loading the trained model
lin_reg = joblib.load('./models/linear_regression_sprint-04.pkl')

In [ ]:
### evaluation
y_test_pred = lin_reg.predict(X_test)

### 7.3. Prediction

#### RMSE

In [ ]:
### computing the final score
from sklearn.metrics import mean_squared_error

lin_rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f'RMSE Lin. Reg. in the Test Set: {lin_rmse_test}')

RMSE Lin. Reg. in the Test Set: 59439.637136249235


By using our linear regression, the **RMSE** for the Test Set -- which has never been seen/used before -- is **\\$ 59,439.63**. <br/>
This error is _slightly higher_ than the _cross-validation error score_ **\\$58,371.04 ± \$1,757.91**, which tends to be common specially when fine-tunning the hyperparameters.

We need now to compare solution with the _current baseline_.

#### Confidence Interval for Squared Errors
In some cases, such a _point estimate_ of the **generalization error** will not be quite enough to convince you to launch: what if it is just _0.1%_ better than the model currently in production? <br/>
You might want to have an idea of how precise this estimate is.

For this, you can compute a ***95% confidence interval*** for the generalization error.

https://github.com/xavecoding/IFSP-CMP-D1AED-2021.1/blob/main/data_distributions/data_distributions.ipynb

<img src='https://github.com/edmgjr/IFSP-CMP-D2APR-2021.2/blob/main/projects/portfolio/project-01_regression_California-housing-prices/imgs/confidence_interval.png?raw=1' />

In [ ]:
confidence_level = 0.95
n = len(y_test)  # the number of testing examples/instances

In [ ]:
squared_errors = (y_test_pred - y_test) ** 2
squared_errors

array([7.28223723e+08, 9.24591190e+07, 2.47612106e+06, ...,
       1.47810096e+09, 5.52327814e+09, 1.45189841e+09])

In [ ]:
standard_error = squared_errors.std() / np.sqrt(n)
standard_error

186474509.66524717

In [ ]:
## alternatively
from scipy import stats
stats.sem(squared_errors, ddof=0)

186474509.66524717

In [ ]:
from scipy.stats import norm

# alpha ==> confidence level
# loc ==> sample mean
# scale ==> standard error

confidence_interval_squared_errors = norm.interval(alpha=confidence_level, loc=squared_errors.mean(), scale=standard_error)
confidence_interval_squared_errors

(3167587139.910329, 3898553785.86763)

In [ ]:
# using the sqrt to keep the erros in the same units
np.sqrt(confidence_interval_squared_errors)

array([56281.32141226, 62438.39993039])

Therefore, we have 95% of confidence that the interval \[\\$56,281.32, \\$62,438.39]\] contains the population generalization error mean.

### 7.4. Comparing our model with the Baseline
Let's first recover the description of the **baseline** from Sprint #1.

#### **Baseline:**
Currently, the **district housing prices** are estimated ***manually by experts***: a team gathers up-to-date information about a district and finds out the _median housing price_. 
This is _costly_ and _time-consuming_, and their **estimates are not great**; they often realize that **their estimates were off by more than 20%**.

Note that this description is a bit vague. We only have an approximation: their estimates were off by more than 20%. <br/>
We do not have a concrete **error** for the baseline. <br/>

To overcome this, we will consider that the baseline estimates final housing prices between **20% and 25% more** than they actually are. 

In [ ]:
np.random.seed(42)

In [ ]:
y_test_pred_baseline = []

for true_housing_price in y_test:
    error_rate = 1 + np.random.randint(20, 26) / 100
    y_test_pred_baseline.append(true_housing_price * error_rate)

#### RMSE

In [ ]:
baseline_rmse_test = mean_squared_error(y_test, y_test_pred_baseline, squared=False)
print(f'RMSE Baseline in the Test Set: {baseline_rmse_test}')

RMSE Baseline in the Test Set: 47911.00494166016


### Discussion

The final performance of our linear regression model (**\\$ 59,439.63**) is not better than the experts’ price estimates (**\\$47,911.00**), which were often off by about 20%. Therefore, it is not prepared to launch in production. We need to find a better model.

We may follow some strategies to find a better model than our current one:
- Evaluate many other different models/algorithms (_e.g.,_ Polynomial regression, KNN regression, SVM regression, ...)
- Apply some feature selection method;
- Perform fine-tunning to find the best hyperparaments
- Try ensemble methods

After all, a model with a score similar to the baseline might be enough. Even though it is not more accurate (or with a lower error) than the baseline, the fact that the model is automatic will frees up some time for the experts so they can work on more interesting and productive tasks.